In [69]:
df = spark.sql("SELECT * FROM fabric_bing_lake_db.tbl_bing_latest_news")
display(df)

StatementMeta(, 6e6fea8a-8d20-499f-b71d-5834992ac735, 71, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 73be954d-40fe-4bd8-9919-999e260b4ee6)

In [70]:
 import synapse.ml.core
 from synapse.ml.services import AnalyzeText

StatementMeta(, 6e6fea8a-8d20-499f-b71d-5834992ac735, 72, Finished, Available, Finished)

In [71]:
#Import the model and configure the input and output columns
model = (AnalyzeText()
         .setTextCol("description")
         .setKind("SentimentAnalysis")
         .setOutputCol("response")
         .setErrorCol("error"))

StatementMeta(, 6e6fea8a-8d20-499f-b71d-5834992ac735, 73, Finished, Available, Finished)

In [72]:
#Apply the model to our dataframe
result = model.transform(df)

StatementMeta(, 6e6fea8a-8d20-499f-b71d-5834992ac735, 74, Finished, Available, Finished)

In [73]:
display(result)

StatementMeta(, 6e6fea8a-8d20-499f-b71d-5834992ac735, 75, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4b18630b-c27f-46a3-9d29-e8fd50919af8)

In [74]:
#Crate Sentiment column
from pyspark.sql.functions import col

sentiment_df = result.withColumn("sentiment", col("response.documents.sentiment"))

StatementMeta(, 6e6fea8a-8d20-499f-b71d-5834992ac735, 76, Finished, Available, Finished)

In [75]:
display(sentiment_df)

StatementMeta(, 6e6fea8a-8d20-499f-b71d-5834992ac735, 77, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c7b23b59-1e18-41bf-b75c-0fd71be94ba2)

In [76]:
sentiment_df_final = sentiment_df.drop("error","response")

StatementMeta(, 6e6fea8a-8d20-499f-b71d-5834992ac735, 78, Finished, Available, Finished)

In [77]:
display(sentiment_df_final)

StatementMeta(, 6e6fea8a-8d20-499f-b71d-5834992ac735, 79, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1669da2e-2835-4502-bc6a-2390efc8dac0)

In [78]:
from pyspark.sql.functions import col, to_date
sentiment_df_final = sentiment_df_final.withColumn("datePublished", to_date(col("datePublished"), "dd-MMM-yyyy"))

StatementMeta(, 6e6fea8a-8d20-499f-b71d-5834992ac735, 80, Finished, Available, Finished)

In [79]:
sentiment_df_final.write.format("delta").mode('overwrite').saveAsTable('fabric_bing_lake_db.tbl_sentiment_analysis')

StatementMeta(, 6e6fea8a-8d20-499f-b71d-5834992ac735, 81, Finished, Available, Finished)

In [80]:
'''from pyspark.sql.utils import AnalysisException
try:

    table_name = 'fabric_bing_lake_db.tbl_sentiment_analysis'

    sentiment_df_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print("Table Already Exists")
   
    sentiment_df_final.createOrReplaceGlobalTempView("vw_sentiment_df_final")
    print('2')

    spark.sql(f""" Merge Into {table_name} target_table
                   Using vw_sentiment_df_final source_view

                   ON source_view.url = target_table.url
                   
                   WHEN MATCHED and
                   source_view.title <> target_table.title OR
                   source_view.description <> target_table.description or
                   source_view.provider <> target_table.provider or
                   source_view.datePublished <> target_table.datePublished
                   
                   THEN UPDATE SET *
                   WHEN NOT MATCHED THEN INSERT *
                 """)'''

StatementMeta(, 6e6fea8a-8d20-499f-b71d-5834992ac735, 82, Finished, Available, Finished)

'from pyspark.sql.utils import AnalysisException\ntry:\n\n    table_name = \'fabric_bing_lake_db.tbl_sentiment_analysis\'\n\n    sentiment_df_final.write.format("delta").saveAsTable(table_name)\n\nexcept AnalysisException:\n\n    print("Table Already Exists")\n   \n    sentiment_df_final.createOrReplaceGlobalTempView("vw_sentiment_df_final")\n    print(\'2\')\n\n    spark.sql(f""" Merge Into {table_name} target_table\n                   Using vw_sentiment_df_final source_view\n\n                   ON source_view.url = target_table.url\n                   \n                   WHEN MATCHED and\n                   source_view.title <> target_table.title OR\n                   source_view.description <> target_table.description or\n                   source_view.provider <> target_table.provider or\n                   source_view.datePublished <> target_table.datePublished\n                   \n                   THEN UPDATE SET *\n                   WHEN NOT MATCHED THEN INSERT *\n       

In [81]:
df = spark.sql("SELECT * FROM fabric_bing_lake_db.tbl_sentiment_analysis LIMIT 1000")
display(df)

StatementMeta(, 6e6fea8a-8d20-499f-b71d-5834992ac735, 83, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cdffe369-653e-4153-bd27-e55cd6fae739)